### Monitoring of the water extent of Lake Neusiedl and the Lacken:

#### Libraries

In [2]:
import os
import glob
from osgeo import gdal
import pprint
import numpy as np
from yeoda.datacube import DataCubeReader
from geopathfinder.naming_conventions.yeoda_naming import YeodaFilename
import pandas as pd
import time
import rasterio
import glob
from rasterio.plot import show
import rasterio
import numpy as np
from rasterio.merge import merge
from rasterio.plot import show
from osgeo import gdal
from data_func import  merge_files
import warnings

In [3]:
warnings.simplefilter(action='ignore', category=FutureWarning)




#### Reading Data

In [4]:
tile_1 = r"./e11739391/Sentinel-1_CSAR/EQUI7_EU010M/E053N015T1"
tile_1 = glob.glob(os.path.join(tile_1, "*"))

In [5]:
tile_2 = r"./e11739391/Sentinel-1_CSAR/EQUI7_EU010M/E052N015T1"
tile_2 = glob.glob(os.path.join(tile_2, "*"))

In [6]:
mask = r"./e11739391/reference/EQUI7_EU010M/E052N015T1"
mask = glob.glob(os.path.join(mask, "*"))

In [12]:
dimensions = ["time", "tile_name", "var_name", "sensor_field", "data_version"]
tile1_reader = DataCubeReader.from_filepaths(tile_1, fn_class=YeodaFilename, dimensions=dimensions,
                                             stack_dimension="time", tile_dimension="tile_name")
tile1_reader.file_register.head(5)

,filepath,var_name,sensor_field,data_version,tile_name,time
0,./e11739391/Sentinel-1_CSAR/EQUI7_EU010M/E053N...,GMR40,S1AIWGRDH,V1M0R01,E053N015T1,2022-09-22 16:42:50
1,./e11739391/Sentinel-1_CSAR/EQUI7_EU010M/E053N...,GMR40,S1AIWGRDH,V1M0R01,E053N015T1,2022-01-17 05:02:29
2,./e11739391/Sentinel-1_CSAR/EQUI7_EU010M/E053N...,GMR40,S1AIWGRDH,V1M0R01,E053N015T1,2022-06-10 05:02:33
3,./e11739391/Sentinel-1_CSAR/EQUI7_EU010M/E053N...,GMR40,S1AIWGRDH,V1M0R01,E053N015T1,2022-04-11 05:02:29
4,./e11739391/Sentinel-1_CSAR/EQUI7_EU010M/E053N...,GMR40,S1AIWGRDH,V1M0R01,E053N015T1,2022-12-07 05:02:12


In [14]:

tile2_reader = DataCubeReader.from_filepaths(tile_2, fn_class=YeodaFilename, dimensions=dimensions,
                                             stack_dimension="time", tile_dimension="tile_name")
tile2_reader.file_register.head(5)

,filepath,var_name,sensor_field,data_version,tile_name,time
0,./e11739391/Sentinel-1_CSAR/EQUI7_EU010M/E052N...,GMR40,S1AIWGRDH,V1M0R01,E052N015T1,2022-06-10 05:02:08
1,./e11739391/Sentinel-1_CSAR/EQUI7_EU010M/E052N...,GMR40,S1AIWGRDH,V1M0R01,E052N015T1,2022-12-07 05:02:37
2,./e11739391/Sentinel-1_CSAR/EQUI7_EU010M/E052N...,GMR40,S1AIWGRDH,V1M0R01,E052N015T1,2022-10-20 05:02:13
3,./e11739391/Sentinel-1_CSAR/EQUI7_EU010M/E052N...,GMR40,S1AIWGRDH,V1M0R01,E052N015T1,2022-02-10 05:02:04
4,./e11739391/Sentinel-1_CSAR/EQUI7_EU010M/E052N...,GMR40,S1AIWGRDH,V1M0R01,E052N015T1,2022-03-06 05:02:03


In [15]:

tile1_reader.file_register.sort_values(by = 'time')['time'].unique()

<DatetimeArray>
['2022-01-17 05:02:04', '2022-01-17 05:02:29', '2022-02-10 05:02:04',
 '2022-02-10 05:02:29', '2022-03-06 05:02:03', '2022-03-06 05:02:28',
 '2022-04-11 05:02:04', '2022-04-11 05:02:29', '2022-05-17 05:02:05',
 '2022-05-17 05:02:30', '2022-06-10 05:02:08', '2022-06-10 05:02:33',
 '2022-07-24 16:42:48', '2022-07-24 16:43:13', '2022-08-05 16:42:49',
 '2022-08-05 16:43:14', '2022-09-22 16:42:50', '2022-09-22 16:43:15',
 '2022-10-20 05:02:13', '2022-10-20 05:02:38', '2022-11-01 05:02:13',
 '2022-11-01 05:02:38', '2022-12-07 05:02:12', '2022-12-07 05:02:37']
Length: 24, dtype: datetime64[ns]

#### Merging files

In [16]:
df_concat = pd.concat([tile1_reader.file_register, tile2_reader.file_register], axis=0)

In [17]:
df_concat.head(5)

,filepath,var_name,sensor_field,data_version,tile_name,time
0,./e11739391/Sentinel-1_CSAR/EQUI7_EU010M/E053N...,GMR40,S1AIWGRDH,V1M0R01,E053N015T1,2022-09-22 16:42:50
1,./e11739391/Sentinel-1_CSAR/EQUI7_EU010M/E053N...,GMR40,S1AIWGRDH,V1M0R01,E053N015T1,2022-01-17 05:02:29
2,./e11739391/Sentinel-1_CSAR/EQUI7_EU010M/E053N...,GMR40,S1AIWGRDH,V1M0R01,E053N015T1,2022-06-10 05:02:33
3,./e11739391/Sentinel-1_CSAR/EQUI7_EU010M/E053N...,GMR40,S1AIWGRDH,V1M0R01,E053N015T1,2022-04-11 05:02:29
4,./e11739391/Sentinel-1_CSAR/EQUI7_EU010M/E053N...,GMR40,S1AIWGRDH,V1M0R01,E053N015T1,2022-12-07 05:02:12


In [18]:
merge_files(df_concat,'VV')

/Users/nadia/PycharmProjects/InterdisciplinaryProject/data_func.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grouped.loc[:, 'date'] = grouped['time'].dt.date


0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.


In [19]:
merge_files(df_concat,'VH')

/Users/nadia/PycharmProjects/InterdisciplinaryProject/data_func.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grouped.loc[:, 'date'] = grouped['time'].dt.date


0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
